# Load C4 models and benchmark while training

In [22]:
from datasets import load_dataset, load_metric
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast, BertForMaskedLM, Trainer, TrainingArguments, BertConfig, DataCollatorForLanguageModeling,  BertTokenizer
from transformers import BertForSequenceClassification
import torch
import torch.nn.functional as F
import numpy as np

In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


In [26]:
# model = BertForMaskedLM.from_pretrained("./bert-c4-results/checkpoint-1240")
# model = BertForSequenceClassification.from_pretrained("./bert-c4-results/checkpoint-1240",  num_labels=2)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",  num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Load GLUE

In [27]:
# Load a specific GLUE task
dataset = load_dataset("glue", "sst2")

In [28]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"], truncation=True, padding="max_length", max_length=256)
    
# Apply the preprocessing function to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [29]:
def compute_metrics(eval_pred):
    metric = load_metric("accuracy")
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return metric.compute(predictions=predictions, references=labels)
    
# Step 5: Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",          # Output directory
    evaluation_strategy="epoch",     # Evaluate at the end of each epoch
    learning_rate=2e-4,              # Learning rate
    per_device_train_batch_size=32,  # Batch size for training
    per_device_eval_batch_size=32,   # Batch size for evaluation
    num_train_epochs=2,              # Number of training epochs
    weight_decay=0.01,               # Weight decay
)

# Step 6: Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,

)

# Step 7: Fine-tune the model
trainer.train()

# Step 8: Evaluate the model
trainer.evaluate()

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.691900,0.712077,0.509174
2,0.685300,0.697019,0.509174


{'eval_loss': 0.697018563747406,
 'eval_accuracy': 0.5091743119266054,
 'eval_runtime': 3.5109,
 'eval_samples_per_second': 248.369,
 'eval_steps_per_second': 7.975,
 'epoch': 2.0}

In [30]:
trainer.evaluate()

{'eval_loss': 0.697018563747406,
 'eval_accuracy': 0.5091743119266054,
 'eval_runtime': 3.5227,
 'eval_samples_per_second': 247.535,
 'eval_steps_per_second': 7.948,
 'epoch': 2.0}

In [31]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# Load the SST-2 dataset
dataset = load_dataset("glue", "sst2")

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Load the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss
1,0.172000,0.260789
2,0.124600,0.308724
3,0.072100,0.325067


{'eval_loss': 0.32506707310676575,
 'eval_runtime': 6.8015,
 'eval_samples_per_second': 128.208,
 'eval_steps_per_second': 8.087,
 'epoch': 3.0}